In [4]:
%matplotlib inline

import numpy as np
import pandas as pd
import gsw
import xarray as xr
from matplotlib import pyplot as plt
from scipy import interpolate

from glob import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import cmocean 

In [5]:
def get_nutricline(crit, nprof, z):
    # use linear interpolation to find depth of nutricline based on critical value
    # first sort the profile so it is descending with depth
    
    f = interpolate.interp1d(nprof, z, 'quadratic')
    znut = f(crit)
    
    if znut <= np.min(z):
        znut = np.nan
    
    return znut

def get_nut_den(crit, nprof, sigma, z):
    # find the density where the critical nitrate value is found
    
    nn = nprof.apply(lambda x: x >= crit)
    
    sig_nut = sigma[nn==True].values.tolist()
    sig_nut = sig_nut[0:4]
    sig_z = z[nn==True].values.tolist()
    sig_z = sig_z[0:4]
    
    if not sig_z:
        sig_nut = np.nan
        sig_z = np.nan
    
    else:
#         print(sig_nut)
#         print(sig_z)
        sig_nut = np.mean(sig_nut[0])
        sig_z = np.mean(sig_z[0])
    
    return np.float64(sig_nut), np.float64(sig_z)


def get_header_line(f, enc, string):
    with open(f,encoding = enc) as myFile:
        for num, line in enumerate(myFile, 1):
            if string in line:
                return num-1
#         f.close()



def get_month(df):
    month = []
    for n in range(len(df)):
        month.append(np.int32(str(df['mon/day/yr'][n]).split('/')[0].lstrip('0   ')))
    return month

def get_casts(df):
    for c in pd.unique(df['mon/day/yr']):
        if not c.empty:
            cast = df[df['mon/day/yr']==c]
            return cast
        
        
def check_lon(data):
    cxx = (data['lon']).values # longitude

    # convert all longitude to longitude east
    cxx[cxx<0]= 360+cxx[cxx<0];
    data['lon_e'] = cxx
    cxbew = cxx
    cxbew[cxbew>180] = cxbew[cxbew>180]-360   
    data['lon'] = cxbew
    return data

In [ ]:
# import QC'd nitrate profiles

infiles = glob('../data/argo/*.TXT')
# infiles = glob('../../kuroshio_productivity/data/argo/*.txt')

column_names = ["lon", "lat", "sig_nut", "sig_z", "mon/day/yr", "float"]
nutc = pd.DataFrame(columns = column_names)

string = "Cruise"
enc = 'cp1252'

fig, ax = plt.subplots(figsize = [9, 6], dpi = 300)

for f in infiles[4:5]:
    num = get_header_line(f, enc, string)
            
    df = pd.read_csv(f, header = num, encoding = enc, sep = '\t', on_bad_lines = 'skip')
    df = df[(df['Nitrate[µmol/kg]'] != -1e10) & (df['QF.8'] == 0)]
#     df = pd.read_csv(f, header = 0)
    crit = 5.0
    
    # counter to keep track of number of casts
    cc = 0 
    ncast = pd.unique(df['mon/day/yr'])
    
    for c in pd.unique(df['mon/day/yr']):
        casts = df[(df['mon/day/yr']==c) & (df['Nitrate[µmol/kg]'] != -1e10) ]
        
        if not casts.empty:
            sig_nut, sig_z = get_nut_den(crit, casts['Nitrate[µmol/kg]'], casts['Sigma_theta[kg/m^3]'], casts['Depth[m]'])
            
            nutc = nutc.append({'mon/day/yr' : c, 'lon' : casts[casts.columns[5]].iloc[0], 'lat' : casts[casts.columns[6]].iloc[0], 
                        'sig_nut': sig_nut, 'sig_z' : sig_z, 'float':casts["Cruise"].iloc[0]}, ignore_index=True)
          
            ax.plot(casts['Nitrate[µmol/kg]']+ cc, casts['Sigma_theta[kg/m^3]'], 'k', alpha = 0.7)
            nn = casts['Nitrate[µmol/kg]'].apply(lambda x: x >= crit)
            ax.plot(casts['Nitrate[µmol/kg]'][nn]+ cc, casts['Sigma_theta[kg/m^3]'][nn], 'r', alpha = 0.7)
            cc += 1
            
    print(f, cc, ' casts, DONE')
    

    ax.invert_yaxis()
    plt.show

    
nutc = check_lon(nutc)
nutc['month'] = get_month(nutc)

nutc['season'] = 'summer'
nutc.loc[(nutc['month'] == 12) | (nutc['month'] == 1) | (nutc['month'] == 2), 'season'] = 'winter'
nutc.loc[(nutc['month'] == 3) | (nutc['month'] == 4) | (nutc['month'] == 5), 'season'] = 'spring'
nutc.loc[(nutc['month'] == 9) | (nutc['month'] == 10) | (nutc['month'] == 11), 'season'] = 'autumn'

# nutc = nutc[(nutc.float != 2902754)]

print("total number of casts:", len(nutc))

In [ ]:
# fig, ax = plt.subplots(figsize = [6, 4], dpi = 300)
# cmap = plt.get_cmap('cmo.phase', 12)

# nut = nutc[(nutc.lat < 34)]#& (nutc.lat < 36)]

# plt.scatter(nut.lon_e, nut.sig_nut, s = 100, c = nut.month, alpha = 0.7, cmap = cmap, vmin = 0.5, vmax = 12.5)
# plt.colorbar()
# plt.show()

cmap = plt.get_cmap('cmo.dense')
cmap = cmocean.tools.crop_by_percent(cmap, 30, which='min', N=None)

fig2, ax2 = plt.subplots(figsize = [6, 4], dpi = 300)
plt.scatter(nut.month, nut.sig_nut, s=50, color = 'k')
plt.xlabel('month')
plt.ylabel('sigma_nutricline')
plt.colorbar()
plt.show()



In [ ]:
print(nutc.groupby("season")[['sig_nut']].median())
print(nutc.groupby("season")[['sig_z']].median())